In [26]:
import pandas as pd

from joblib import load
# Tools to remove stopwords from tweets
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from sklearn.linear_model import SGDClassifier

[nltk_data] Downloading package stopwords to C:\Users\Herman D
[nltk_data]     Schaumburg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Herman D
[nltk_data]     Schaumburg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
# Load R or D predictor
bigram_vectorizer = load('data_preprocessors/bigram_vectorizer.joblib')
bigram_tf_idf_transformer = load('data_preprocessors/bigram_tf_idf_transformer.joblib')
bigram_vectorizer = load('data_preprocessors/bigram_vectorizer.joblib')
bigram_tf_idf_transformer = load('data_preprocessors/bigram_tf_idf_transformer.joblib')

In [28]:
stop_words = set(stopwords.words('english'))

def fix_party_code(pc):
    if pc == 100:
        return 'D'
    else:
        return 'R'
#    return int(pc/100-1)
def list_tostring(input_list):
    return ' '.join(input_list)

def remove_stopwords(input_list):
    return [w for w in input_list if not w in stop_words]

def clean_tweets(input_df):
    input_df['party_code'] = input_df['party_code'].apply(fix_party_code)
    input_df['text'] = input_df['text'].apply(word_tokenize)
    input_df['text'] = input_df['text'].apply(remove_stopwords)
    input_df['text'] = input_df['text'].apply(list_tostring)    
    return input_df

In [29]:
congress_df = pd.read_json("HS116_members.json")
congress_df.head()

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,...,died,nominate_dim1,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2
0,116,House,20301,41,3,AL,200,,,"ROGERS, Mike Dennis",...,NaN,0.353,0.459,-157.12696,0.78556,651,64,NaN,0.520,0.347
1,116,House,21102,41,7,AL,100,,,"SEWELL, Terri",...,NaN,-0.391,0.402,-26.01682,0.96146,662,11,NaN,-0.435,0.395
2,116,House,21192,41,2,AL,200,,,"ROBY, Martha",...,NaN,0.361,0.656,-86.62113,0.87433,645,30,NaN,0.341,0.673
3,116,House,21193,41,5,AL,200,,,"BROOKS, Mo",...,NaN,0.647,-0.446,-124.80360,0.83187,678,50,NaN,0.786,-0.368
4,116,House,21376,41,1,AL,200,,,"BYRNE, Bradley",...,NaN,0.611,0.244,-101.01760,0.84830,614,40,NaN,0.707,0.167


In [30]:
party_code_df = pd.read_csv("partycode.csv")
party_code_df.head()

,account_id,bioguide_id,mem_name,screen_name,chamber,state_abbr,party_code
0,37007274,Y000033,Don Young,repdonyoung,House,AK,200
1,2559398984,Y000033,Don Young,DonYoungAK,House,AK,200
2,2253968388,B001289,Bradley Byrne,RepByrne,House,AL,200
3,42481696,B001289,Bradley Byrne,BradleyByrne,House,AL,200
4,2861616083,P000609,Gary Palmer,USRepGaryPalmer,House,AL,200


In [31]:
# Identify congresspeople by their twitter IDs
twit_congress_df = congress_df.merge(party_code_df[['bioguide_id','account_id','mem_name']], how = 'right', left_on='bioguide_id', right_on='bioguide_id')

In [32]:
# Verify multiple twitter accounts for congressperson
twit_congress_df[twit_congress_df['mem_name']=='Bradley Byrne']

,congress,chamber,icpsr,state_icpsr,district_code,state_abbrev,party_code,occupancy,last_means,bioname,...,nominate_dim2,nominate_log_likelihood,nominate_geo_mean_probability,nominate_number_of_votes,nominate_number_of_errors,conditional,nokken_poole_dim1,nokken_poole_dim2,account_id,mem_name
6,116,House,21376,41,1,AL,200,,,"BYRNE, Bradley",...,0.244,-101.0176,0.8483,614,40,NaN,0.707,0.167,2253968388,Bradley Byrne
7,116,House,21376,41,1,AL,200,,,"BYRNE, Bradley",...,0.244,-101.0176,0.8483,614,40,NaN,0.707,0.167,42481696,Bradley Byrne


In [33]:
twit_congress_df[['account_id','party_code','nominate_dim1','nominate_dim2']]

,account_id,party_code,nominate_dim1,nominate_dim2
0,381152398,100,-0.391,0.402
1,69326168,100,-0.391,0.402
2,224294785,200,0.361,0.656
3,69086398,200,0.361,0.656
4,237299871,200,0.647,-0.446
...,...,...,...,...
1080,142332083,200,0.603,-0.288
1081,202206694,200,0.539,0.237
1082,499268312,200,0.539,0.237
1083,291756142,200,0.544,0.192


In [34]:
twit_id_dict = twit_congress_df[['account_id','nominate_dim1','nominate_dim2','party_code']].set_index('account_id').to_dict()
twit_id_dict['nominate_dim1'][69326168]

-0.391

In [35]:
# Specify dates for training data
dates = pd.date_range(start='4/1/2020', end='4/30/2020')
dates = [str(date)[0:10] for date in dates]

# Specify number of samples per day per party
date_samples = 1130
print('Training day consists of '+str(2*date_samples*0.75*len(dates))+' tweets.')

Training day consists of 50850.0 tweets.


In [36]:
# Create empty dataframe
train_df = pd.DataFrame(columns=['text','nominate_dim1'])

for date in dates:
    date_str = date
    example_tweets = pd.read_json("mediabias_main/congresstweets/data/"+date_str+".json")
    example_tweets["nominate_dim1"] = example_tweets["user_id"]
    example_tweets["party_code"] = example_tweets["user_id"]
    example_tweets = example_tweets.replace({"nominate_dim1":twit_id_dict['nominate_dim1']})
    example_tweets = example_tweets.replace({"party_code":twit_id_dict['party_code']})
    date_train_data = example_tweets[(example_tweets['party_code']!=100)|(example_tweets['party_code']!=200)][['text','party_code','nominate_dim1']]
    # Rearrange rows
    date_train_data = date_train_data.sample(frac=1)
    date_train_data_R=date_train_data[date_train_data['party_code']==200].head(date_samples)
    date_train_data_D=date_train_data[date_train_data['party_code']==100].head(date_samples)
    # Remove stop words and fix party code
    date_train_data = clean_tweets(date_train_data_D.append(date_train_data_R, ignore_index=True))
    date_train_data = date_train_data[['text','nominate_dim1']]
    #if date == date[0]:
    #    train_df = date_train_data
    #else:
    train_df = train_df.append(date_train_data)#, ignore_index=True)

In [37]:
train_df = train_df.reset_index(drop=True)

# Text Vectorization - mixed

In [38]:
from os import system
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from joblib import dump, load # used for saving and loading sklearn objects
from scipy.sparse import save_npz, load_npz # used for saving and loading sparse matrices
# from sklearn.preprocessing import StandardScaler

In [39]:
system("mkdir data_preprocessors")
system("mkdir vectorized_data")

# Unigram Counts
unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
unigram_vectorizer.fit(train_df['text'].values)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [40]:
dump(unigram_vectorizer, 'data_preprocessors/unigram_vectorizer-nom-mixed.joblib')

['data_preprocessors/unigram_vectorizer-nom-mixed.joblib']

In [41]:
# unigram_vectorizer = load('data_preprocessors/unigram_vectorizer.joblib')

X_train_unigram = unigram_vectorizer.transform(train_df['text'].values)
save_npz('vectorized_data/X_train_unigram-nom-mixed.npz', X_train_unigram)

In [42]:
# Unigram Tf-Idf

unigram_tf_idf_transformer = TfidfTransformer()
unigram_tf_idf_transformer.fit(X_train_unigram)

dump(unigram_tf_idf_transformer, 'data_preprocessors/unigram_tf_idf_transformer-non-mixed.joblib')

['data_preprocessors/unigram_tf_idf_transformer-non-mixed.joblib']

In [43]:
X_train_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_train_unigram)

save_npz('vectorized_data/X_train_unigram_tf_idf-non-mixed.npz', X_train_unigram_tf_idf)

In [44]:
# Bigram Counts

bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))
bigram_vectorizer.fit(train_df['text'].values)

dump(bigram_vectorizer, 'data_preprocessors/bigram_vectorizer-non-mixed.joblib')

['data_preprocessors/bigram_vectorizer-non-mixed.joblib']

In [45]:
X_train_bigram = bigram_vectorizer.transform(train_df['text'].values)

save_npz('vectorized_data/X_train_bigram-non-mixed.npz', X_train_bigram)

In [46]:
bigram_tf_idf_transformer = TfidfTransformer()
bigram_tf_idf_transformer.fit(X_train_bigram)

dump(bigram_tf_idf_transformer, 'data_preprocessors/bigram_tf_idf_transformer-non-mixed.joblib')

['data_preprocessors/bigram_tf_idf_transformer-non-mixed.joblib']

In [47]:
X_train_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_train_bigram)

save_npz('vectorized_data/X_train_bigram_tf_idf-non-mixed.npz', X_train_bigram_tf_idf)

# SGDRegressor - Mixed

In [48]:
def train_and_show_scores(X: csr_matrix, y: np.array, title: str) -> None:
    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, train_size=0.75
    )

    clf = SGDRegressor(max_iter=2000, tol=1e-5)
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    valid_score = clf.score(X_valid, y_valid)
    print(f'{title}\nTrain score: {round(train_score, 2)} ; Validation score: {round(valid_score, 2)}\n')

y_train = train_df['nominate_dim1'].values

In [49]:
train_and_show_scores(X_train_unigram, y_train, 'Unigram Counts')
train_and_show_scores(X_train_unigram_tf_idf, y_train, 'Unigram Tf-Idf')
train_and_show_scores(X_train_bigram, y_train, 'Bigram Counts')
train_and_show_scores(X_train_bigram_tf_idf, y_train, 'Bigram Tf-Idf')

Unigram Counts
Train score: 0.79 ; Validation score: 0.51

Unigram Tf-Idf
Train score: 0.53 ; Validation score: 0.46

Bigram Counts
Train score: 0.95 ; Validation score: 0.55

Bigram Tf-Idf
Train score: 0.53 ; Validation score: 0.42



# Seperate by party

In [50]:
# Specify dates for training data
dates = pd.date_range(start='4/1/2020', end='4/30/2020')
dates = [str(date)[0:10] for date in dates]

# Specify number of samples per day per party
date_samples = 1130
print('Training day consists of '+str(date_samples*0.75*len(dates))+' tweets.')

Training day consists of 25425.0 tweets.


In [51]:
# Create empty dataframe
R_train_df = pd.DataFrame(columns=['text','nominate_dim1'])
D_train_df = pd.DataFrame(columns=['text','nominate_dim1'])

for date in dates:
    date_str = date
    example_tweets = pd.read_json("mediabias_main/congresstweets/data/"+date_str+".json")
    example_tweets["nominate_dim1"] = example_tweets["user_id"]
    example_tweets["party_code"] = example_tweets["user_id"]
    example_tweets = example_tweets.replace({"nominate_dim1":twit_id_dict['nominate_dim1']})
    example_tweets = example_tweets.replace({"party_code":twit_id_dict['party_code']})
    date_train_data = example_tweets[(example_tweets['party_code']!=100)][['text','party_code','nominate_dim1']]
    date_train_data = date_train_data[['text','nominate_dim1']]
    R_train_df = train_df.append(date_train_data)#, ignore_index=True)
    
    date_train_data = example_tweets[(example_tweets['party_code']!=200)][['text','party_code','nominate_dim1']]
    date_train_data = date_train_data[['text','nominate_dim1']]
    D_train_df = train_df.append(date_train_data)#, ignore_index=True)

In [52]:
R_train_df = R_train_df.reset_index(drop=True)
D_train_df = D_train_df.reset_index(drop=True)

In [53]:
# Unigram Counts
unigram_vectorizer = CountVectorizer(ngram_range=(1, 1))
unigram_vectorizer.fit(R_train_df['text'].values)
dump(unigram_vectorizer, 'data_preprocessors/unigram_vectorizer-nom-R.joblib')
unigram_vectorizer.fit(D_train_df['text'].values)
dump(unigram_vectorizer, 'data_preprocessors/unigram_vectorizer-nom-D.joblib')

KeyboardInterrupt: 

In [ ]:
X_R_train_unigram = unigram_vectorizer.transform(R_train_df['text'].values)
save_npz('vectorized_data/X_train_unigram-nom-mixed.npz', X_R_train_unigram)

X_D_train_unigram = unigram_vectorizer.transform(D_train_df['text'].values)
save_npz('vectorized_data/X_train_unigram-nom-mixed.npz', X_D_train_unigram)

In [ ]:
# in_text = request.form['text']
# fun_input = list_tostring(word_tokenize(in_text))

# X_pred = bigram_vectorizer.transform([fun_input])
# X_pred = bigram_tf_idf_transformer.transform(X_pred)
# sgd_classifier = load('classifiers/sgd_classifier.joblib')
# result = sgd_classifier.predict(X_pred)